In [1]:
import os
import json
import GEOparse
import pandas as pd
from openai import OpenAI

from prompts.column_description import system_message, dataset_columns_description_prompt, dataset_input
from utils import utils
from utils import openai_utils
import prompts
from importlib import reload

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
reload(utils)
reload(openai_utils)
reload(prompts.column_description)

<module 'prompts.column_description' from '/home/bek/Desktop/iCog/semantic parsing/Bio_data_semantic_parsing/prompts/column_description.py'>

In [3]:
gse= GEOparse.get_GEO(geo="GSE12277", destdir="./data/GEO")

05-Mar-2025 14:36:04 DEBUG utils - Directory ./data/GEO already exists. Skipping.
05-Mar-2025 14:36:04 INFO GEOparse - File already exist: using local version.
05-Mar-2025 14:36:04 INFO GEOparse - Parsing ./data/GEO/GSE12277_family.soft.gz: 
05-Mar-2025 14:36:04 DEBUG GEOparse - DATABASE: GeoMiame
05-Mar-2025 14:36:04 DEBUG GEOparse - SERIES: GSE12277
05-Mar-2025 14:36:04 DEBUG GEOparse - PLATFORM: GPL570
/home/bek/Desktop/iCog/semantic parsing/Bio_data_semantic_parsing/venv/lib/python3.12/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
05-Mar-2025 14:36:10 DEBUG GEOparse - SAMPLE: GSM308237
05-Mar-2025 14:36:11 DEBUG GEOparse - SAMPLE: GSM308239
05-Mar-2025 14:36:11 DEBUG GEOparse - SAMPLE: GSM308241
05-Mar-2025 14:36:11 DEBUG GEOparse - SAMPLE: GSM308242
05-Mar-2025 14:36:12 DEBUG GEOparse - SAMPLE: GSM308244
05-Mar-2025 14:36:12 DEBU

### Column description generation for the table in GSMs

In [5]:
gsm= gse.gsms["GSM308246"]
data= gsm.table
data.head()

,ID_REF,VALUE,ABS_CALL,DETECTION P-VALUE
0,AFFX-BioB-5_at,115.4570,P,0.000581
1,AFFX-BioB-M_at,119.7700,P,0.000044
2,AFFX-BioB-3_at,84.8163,P,0.000095
3,AFFX-BioC-5_at,296.3510,P,0.000052
4,AFFX-BioC-3_at,470.6520,P,0.000044


In [5]:
sample = data.sample(20)

In [6]:
df_string = sample.to_string()
print(df_string)


            ID_REF       VALUE ABS_CALL  DETECTION P-VALUE
43515    243653_at    1.192690        A           0.919434
14012  214572_s_at   27.373000        P           0.010742
10016  210479_s_at    5.621650        A           0.246094
5513   205925_s_at    0.630271        A           0.696289
49939   1560019_at    0.565803        A           0.870361
52678   1565617_at   34.498600        A           0.129639
14119    214682_at    1.148910        A           0.633789
32982    233116_at   58.970800        P           0.023926
42847  242985_x_at    8.164640        A           0.466064
24362  224488_s_at    1.151420        A           0.432373
24049    224164_at   38.744500        P           0.001953
3863     204275_at   45.129000        P           0.001953
50574   1561093_at   39.578400        A           0.194580
36659    236797_at    3.531530        A           0.274170
17823    218397_at  471.524000        P           0.000244
22713  222821_s_at  122.475000        P           0.0185

In [7]:
dataset_description= "GEO is a public functional genomics data repository supporting MIAME-compliant data submissions. Array- and sequence-based data are accepted. Tools are provided to help users query and download experiments and curated gene expression profiles."

In [12]:
generation_params = {
    'temperature': 0.0,
    'max_tokens': 4096,
}

In [9]:
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': dataset_columns_description_prompt 
                                + dataset_input.format(dataset_description= dataset_description, dataset= df_string)}
]

In [10]:
response = openai_utils.openai_generate(messages=messages, **generation_params)
descriptions = utils.postprocess_response(response.choices[0].message.content)

In [11]:
descriptions

{'ID_REF': {'description': 'Unique identifier for each gene expression profile in the dataset.',
  'data_type': 'string'},
 'VALUE': {'description': 'Quantitative value representing the gene expression level.',
  'data_type': 'float'},
 'ABS_CALL': {'description': "Categorical value indicating the quality of the measurement, with 'A' indicating absent, 'P' indicating present, and 'M' indicating marginal.",
  'data_type': 'string'},
 'DETECTION P-VALUE': {'description': 'Statistical value indicating the reliability of the measurement, with lower values indicating higher reliability.',
  'data_type': 'float'}}

### Predicate generation for table entries

In [22]:
gsm_id = gsm.metadata.get("geo_accession", ["Unknown_GSE"])[0]
print(gsm_id)

GSM308246


Generate a unique ID for each row in a GSM

In [24]:

# Create a unique ID by combining row index and GSM ID
data["Unique_ID"] = data.index.astype(str) + "_" + gsm_id

# Display the updated DataFrame
print(data.head(-5))

               ID_REF       VALUE ABS_CALL  DETECTION P-VALUE        Unique_ID
0      AFFX-BioB-5_at  115.457000        P           0.000581      0_GSM308246
1      AFFX-BioB-M_at  119.770000        P           0.000044      1_GSM308246
2      AFFX-BioB-3_at   84.816300        P           0.000095      2_GSM308246
3      AFFX-BioC-5_at  296.351000        P           0.000052      3_GSM308246
4      AFFX-BioC-3_at  470.652000        P           0.000044      4_GSM308246
...               ...         ...      ...                ...              ...
54665      1570631_at   12.766800        P           0.023926  54665_GSM308246
54666      1570632_at    1.179860        A           0.780518  54666_GSM308246
54667      1570633_at    0.159784        A           0.953857  54667_GSM308246
54668      1570635_at    1.926470        A           0.753906  54668_GSM308246
54669      1570639_at    4.649640        A           0.398926  54669_GSM308246

[54670 rows x 5 columns]


In [26]:
sample_data=data.sample(10)
print(sample_data) 

             ID_REF       VALUE ABS_CALL  DETECTION P-VALUE        Unique_ID
27172     227304_at   69.489700        P           0.000244  27172_GSM308246
1888    202299_s_at  603.042000        P           0.000244   1888_GSM308246
54150    1569736_at    3.065860        A           0.129639  54150_GSM308246
30820     230953_at    1.205890        A           0.850342  30820_GSM308246
35723     235861_at   28.253600        P           0.000244  35723_GSM308246
47739  1556341_s_at   62.745900        A           0.111572  47739_GSM308246
35973     236111_at    1.477930        A           0.466064  35973_GSM308246
4984      205396_at   11.996100        A           0.246094   4984_GSM308246
19616     220191_at    0.199738        A           0.601074  19616_GSM308246
31123     231256_at    8.607630        A           0.303711  31123_GSM308246


In [27]:
columns = list(sample_data.columns)
print("Extracted Columns:", columns)


Extracted Columns: ['ID_REF', 'VALUE', 'ABS_CALL', 'DETECTION P-VALUE', 'Unique_ID']


In [28]:
prompt = f"""
    You are a data standardization expert. Convert the following dataset column names into standardized predicate names:
    {columns}
    Return a JSON dictionary where keys are original column names and values are standardized predicate names.
    """

In [29]:
messages =[
    {'role':'system', 'content':"You are an expert in data standardization."},
    {'role':'user', 'content':prompt}
]

In [41]:
response = openai_utils.openai_generate(messages=messages, **generation_params)
predicate_mapping = response.choices[0].message.content

In [42]:
print("Predicate Mapping:", predicate_mapping)

Predicate Mapping: {
    "ID_REF": "idRef",
    "VALUE": "value",
    "ABS_CALL": "absCall",
    "DETECTION P-VALUE": "detectionPValue",
    "Unique_ID": "uniqueId"
}


In [43]:
predicate_mapping = json.loads(predicate_mapping)

In [69]:
def generate_fol_predicates(row, predicate_mapping):
    """Convert a row of data into FOL predicates."""
    unique_id = row["Unique_ID"]  # Use Unique_ID as the main entity
    predicates = [f'GSM_Measurement({unique_id})']  # Main predicate

    # Convert each column into a predicate
    for col, predicate in predicate_mapping.items():
        value = row[col]
        formatted_value = value if pd.notna(value) else "null"

         # Exclude uniqueId(Unique_ID, Unique_ID)
        if predicate == "uniqueId" and formatted_value == unique_id:
            continue
        
        predicates.append(f'{predicate}({unique_id}, {formatted_value})')

    return ",\n    ".join(predicates) +","

In [70]:
# Generate predicates for each row
fol_predicates = [generate_fol_predicates(row, predicate_mapping) for _, row in sample_data.iterrows()]

# Print results
for pred in fol_predicates:
    print(pred)




GSM_Measurement(27172_GSM308246),
    idRef(27172_GSM308246, 227304_at),
    value(27172_GSM308246, 69.4897),
    absCall(27172_GSM308246, P),
    detectionPValue(27172_GSM308246, 0.000244141),
GSM_Measurement(1888_GSM308246),
    idRef(1888_GSM308246, 202299_s_at),
    value(1888_GSM308246, 603.042),
    absCall(1888_GSM308246, P),
    detectionPValue(1888_GSM308246, 0.000244141),
GSM_Measurement(54150_GSM308246),
    idRef(54150_GSM308246, 1569736_at),
    value(54150_GSM308246, 3.06586),
    absCall(54150_GSM308246, A),
    detectionPValue(54150_GSM308246, 0.129639),
GSM_Measurement(30820_GSM308246),
    idRef(30820_GSM308246, 230953_at),
    value(30820_GSM308246, 1.20589),
    absCall(30820_GSM308246, A),
    detectionPValue(30820_GSM308246, 0.850342),
GSM_Measurement(35723_GSM308246),
    idRef(35723_GSM308246, 235861_at),
    value(35723_GSM308246, 28.2536),
    absCall(35723_GSM308246, P),
    detectionPValue(35723_GSM308246, 0.000244141),
GSM_Measurement(47739_GSM308246),
    

In [71]:
with open("./outputs/predicates.txt", "w") as file:
    file.write("\n".join(fol_predicates))

### Generating predicates that represent the relationship between GSE and GSM

In [56]:
gse.metadata

{'title': ['Hematopoietic Progenitor Cells of Different Donor Age'],
 'geo_accession': ['GSE12277'],
 'status': ['Public on Jun 12 2009'],
 'submission_date': ['Jul 29 2008'],
 'last_update_date': ['Mar 25 2019'],
 'pubmed_id': ['19513108'],
 'summary': ['In this series we have analyzed the effect of donor age on the gene expression profile of human hematopoietic stem and progenitor cells (HPC). Cells were taken from umbilical cord blood (CB) or from G-CSF mobilized blood of healthy donors for allogeneic blood stem cell transplantation.'],
 'overall_design': ['Hematopoietic progenitor cells (HPC) were isolated within the CD34+ cell fraction from fresh human cord blood (CB) or from G-CSF mobilized peripheral blood (PB) as described in detail before (Wagner et al., Blood, 2004, 104:675-684; Wagner et al., Stem cells, 2005, 23:1180-1191; Wagner et al., 2007, 10:2638-2657). Differential gene expression was subsequently compared in the CD34+ samples of different donor age.'],
 'type': ['Exp

In [57]:
def generate_fol_relationship(metadata):
    """
    Generates First-Order Logic (FOL) representation of the GSE-GSM relationship.

    Args:
        metadata (dict): Dictionary containing GSE metadata.

    Returns:
        str: FOL statements representing GSE-GSM relationships.
    """
    gse_id = metadata.get("geo_accession", ["Unknown_GSE"])[0]  # Get GSE ID
    gsm_ids = metadata.get("sample_id", [])  # Get GSM IDs list

    fol_statements = [f"has_sample({gse_id}, {gsm});" for gsm in gsm_ids]  # Create FOL statements

    return "\n".join(fol_statements)

In [58]:
gse_metadata= gse.metadata

In [59]:
fol_output = generate_fol_relationship(gse_metadata)

In [60]:
print("Example FOL Output:")
print(fol_output)

Example FOL Output:
has_sample(GSE12277, GSM308237);
has_sample(GSE12277, GSM308239);
has_sample(GSE12277, GSM308241);
has_sample(GSE12277, GSM308242);
has_sample(GSE12277, GSM308244);
has_sample(GSE12277, GSM308245);
has_sample(GSE12277, GSM308246);
has_sample(GSE12277, GSM308247);
has_sample(GSE12277, GSM308254);
has_sample(GSE12277, GSM308255);
has_sample(GSE12277, GSM308317);
has_sample(GSE12277, GSM308461);
has_sample(GSE12277, GSM308462);
has_sample(GSE12277, GSM308463);
has_sample(GSE12277, GSM308465);
has_sample(GSE12277, GSM308467);
has_sample(GSE12277, GSM308469);
has_sample(GSE12277, GSM308470);
has_sample(GSE12277, GSM308473);


### PLN formatted predicate generation